## Essay Evaluator

In [134]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from typing import TypedDict, Annotated
from pydantic import BaseModel, Field
import operator

In [135]:
load_dotenv()

True

In [136]:
model = ChatOpenAI(model='gpt-4o-mini')

In [137]:
class EvaluationSchema(BaseModel):

    feedback: str = Field(description='Detailed feedback for the essay')
    score: int = Field(description='Score out of 10', ge=0, le=10)

In [138]:
structured_model = model.with_structured_output(EvaluationSchema)

In [139]:
essay = """Building Complex Agentic AI Workflows with LangGraph

LangGraph, an extension of the LangChain ecosystem, introduces a powerful framework for constructing sophisticated, stateful AI workflows. By representing each computational step as a node within a directed graph, LangGraph enables the design of dynamic, multi-agent systems capable of handling complex decision-making processes.

What Is LangGraph?

LangGraph transforms traditional linear AI pipelines into flexible, graph-based architectures. Each node in the graph represents a distinct task—such as information retrieval, data processing, or decision-making—while the edges define the flow of information and control between these tasks. This structure allows for the creation of workflows where the sequence of operations can adapt based on conditions, enabling more intelligent and autonomous behavior.

Core Features of LangGraph

Stateful Execution: Nodes maintain their own state, allowing for persistent memory across the workflow.

Dynamic Routing: Edges can be conditionally activated, facilitating complex branching logic.

Modular Design: Workflows can be composed of reusable subgraphs, promoting scalability and maintainability.

Multi-Agent Support: Multiple independent agents can be orchestrated within a single workflow, each with its own prompt, tools, and logic.

Building Complex Agentic Workflows

LangGraph's architecture is particularly suited for developing agentic AI systems, where multiple agents collaborate to achieve a common goal. Each agent operates autonomously, making decisions based on its inputs and the overall state of the system. This approach is ideal for tasks that require specialized knowledge, iterative reasoning, or human-in-the-loop interactions.

Example: Multi-Agent Collaboration

In a multi-agent setup, each agent can be tasked with a specific role, such as data extraction, analysis, or reporting. These agents communicate through shared states or messages, coordinating their efforts to produce a cohesive output. For instance, one agent might gather data from various sources, while another analyzes the data and a third generates a report. The orchestration of these agents ensures that each step is performed in the correct sequence and under appropriate conditions.

Example: Agent Supervisor

An agent supervisor acts as a central controller, managing the flow of tasks between multiple agents. It decides which agent to invoke based on the current state and the desired outcome. This setup is useful for workflows that require oversight or decision-making at a higher level, ensuring that the right resources are applied at the right time.

Practical Applications

LangGraph's capabilities make it suitable for a wide range of applications:

Research Assistants: Agents that autonomously gather, analyze, and summarize information from various sources.

Automated Reporting: Systems that generate reports by collecting data, performing analyses, and compiling findings into structured documents.

Customer Support: AI agents that handle customer inquiries by retrieving relevant information, formulating responses, and escalating issues when necessary.

Data Processing Pipelines: Workflows that process and transform data through a series of specialized agents, each performing a specific task.

Getting Started with LangGraph

To begin building workflows with LangGraph, you'll need to install the necessary packages:

pip install langchain langgraph


Once installed, you can start defining your agents and workflows using LangGraph's API, which provides tools for creating nodes, defining edges, and managing state transitions. The framework also offers visualization tools to help you design and debug your workflows effectively.

Conclusion

LangGraph empowers developers to construct complex, agent-based AI systems that are both flexible and scalable. By leveraging its graph-based architecture, you can create workflows that are not only intelligent and autonomous but also adaptable to changing conditions and requirements. Whether you're building a research assistant, an automated reporting system, or a customer support agent, LangGraph provides the tools necessary to bring your vision to life."""

In [140]:
# Prompt Testing
prompt = f'Evaluate the language quality of the following essay and provide a feedback and assign a score out of 10 \n {essay}'
print(structured_model.invoke(prompt).feedback)
print(structured_model.invoke(prompt).score)

The essay on LangGraph is well-structured, providing a clear introduction, in-depth explanations of key features, examples, practical applications, and a strong conclusion. The language used is formal and technical, suitable for an audience familiar with AI concepts. It effectively communicates complex ideas through well-defined sections, allowing readers to understand the significance of LangGraph in AI development. However, there are areas to improve: some sentences are quite dense and could be simplified for better readability. Additionally, introducing a brief overview of the LangChain ecosystem at the beginning could provide better context for readers unfamiliar with it. Overall, the essay demonstrates a solid grasp of the subject matter and articulates concepts clearly. Consider improving the flow of information and simplifying complex sentences to enhance clarity and engagement.
8


In [141]:
class EssayState(TypedDict):
    essay:str
    language_feedback: str
    analysis_feedback: str
    clarity_feedback: str
    overall_feedback: str
    language_score: int
    analysis_score: int
    clarity_score: int
    overall_score: float

In [142]:
def evaluate_language(state: EssayState):

    prompt = f'Evaluate the language quality of the following essay and provide a feedback and assign a score out of 10 \n {state["essay"]}'
    output = structured_model.invoke(prompt)
    print("Language Feedback: \n" + output.feedback)
    print(f'Language Score: \n  {output.score}')

    return {'language_feedback': output.feedback, 'language_score': output.score}

In [143]:
def evaluate_analysis(state: EssayState):

    prompt = f'Evaluate the depth of analysis of the following essay and provide a feedback and assign a score out of 10 \n {state["essay"]}'
    output = structured_model.invoke(prompt)
    print("Analysis Feedback: \n" + output.feedback)
    print(f'Analysis Score: \n  {output.score}')

    return {'analysis_feedback': output.feedback, 'analysis_score': output.score}

In [144]:
def evaluate_thought(state: EssayState):

    prompt = f'Evaluate the clarity of thought of the following essay and provide a feedback and assign a score out of 10 \n {state["essay"]}'
    output = structured_model.invoke(prompt)
    print("Thought Feedback: \n" + output.feedback)
    print(f'Thought Score: \n  {output.score}')

    return {'clarity_feedback': output.feedback, 'clarity_score': output.score}

In [145]:
def final_evaluation(state: EssayState):
    # summary feedback
    prompt = f"""Based on the following feedbacks, create a summarized feedback:
    Language feedback: {state['language_feedback']}
    Depth of analysis feedback: {state['analysis_feedback']}
    Clarity of thought feedback: {state['clarity_feedback']}
    """
    overall_feedback = model.invoke(prompt).content
    print("Overall Feedback: \n" + overall_feedback)

    # average score calculation
    avg_score = (state['analysis_score'] + state['clarity_score'] + state['language_score']) / 3
    print(f'Overall Score: \n  {avg_score}')

    return {'overall_feedback': overall_feedback, 'overall_score': avg_score}

In [146]:
graph = StateGraph(EssayState)

graph.add_node('evaluate_language', evaluate_language)
graph.add_node('evaluate_analysis', evaluate_analysis)
graph.add_node('evaluate_thought', evaluate_thought)
graph.add_node('final_evaluation', final_evaluation)

# edges
graph.add_edge(START, 'evaluate_language')
graph.add_edge(START, 'evaluate_analysis')
graph.add_edge(START, 'evaluate_thought')

graph.add_edge('evaluate_language', 'final_evaluation')
graph.add_edge('evaluate_analysis', 'final_evaluation')
graph.add_edge('evaluate_thought', 'final_evaluation')

graph.add_edge('final_evaluation', END)

workflow = graph.compile()

In [ ]:
workflow

In [148]:
intial_state = {
    'essay': essay
}

workflow.invoke(intial_state)

Thought Feedback: 
The essay presents a clear and coherent explanation of LangGraph, effectively outlining its features, applications, and how it differs from traditional AI workflows. The structure is logical, with distinct sections that build upon one another, making the information accessible to both tech-savvy readers and those less familiar with AI workflows. Concepts such as stateful execution, dynamic routing, and modular design are well-defined, and the examples provided enhance understanding.

However, the essay could benefit from a slight increase in clarity for readers unfamiliar with the concepts, perhaps by briefly defining jargon terms or providing analogies. Additionally, the introduction could include the broader implications or significance of LangGraph in the evolving AI landscape, which might engage the reader more effectively. 

Overall, the essay demonstrates strong clarity of thought and well-organized content, making it an informative read on LangGraph.
Thought S

{'essay': "Building Complex Agentic AI Workflows with LangGraph\n\nLangGraph, an extension of the LangChain ecosystem, introduces a powerful framework for constructing sophisticated, stateful AI workflows. By representing each computational step as a node within a directed graph, LangGraph enables the design of dynamic, multi-agent systems capable of handling complex decision-making processes.\n\nWhat Is LangGraph?\n\nLangGraph transforms traditional linear AI pipelines into flexible, graph-based architectures. Each node in the graph represents a distinct task—such as information retrieval, data processing, or decision-making—while the edges define the flow of information and control between these tasks. This structure allows for the creation of workflows where the sequence of operations can adapt based on conditions, enabling more intelligent and autonomous behavior.\n\nCore Features of LangGraph\n\nStateful Execution: Nodes maintain their own state, allowing for persistent memory acr